In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import datetime
import time

In [2]:
#Defining the size of the grid
n_cell_x = 20
n_cell_y = 40 

def cells_around(center):
    cells = []
    for i in range(-1, 2):
        for j in range(-1, 2):
            cells.append(center + j * n_cell_x + i)
    return cells

def prepare_data(df, n_cell_x, n_cell_y):
    """
    Feature engineering and computation of the grid.
    """
    #Creating the grid
    size_x = 10. / n_cell_x
    size_y = 10. / n_cell_y
    eps = 0.00001  
    xs = np.where(df.x.values < eps, 0, df.x.values - eps)
    ys = np.where(df.y.values < eps, 0, df.y.values - eps)
    pos_x = (xs / size_x).astype(np.int)
    pos_y = (ys / size_y).astype(np.int)
    df['grid_cell'] = pos_y * n_cell_x + pos_x
    
    initial_date = np.datetime64('2014-01-01T01:01', dtype='datetime64[m]') 
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df.time.values)    
    df['hour'] = (d_times.hour + d_times.minute/60)
    df['weekday'] = d_times.weekday
    df['day'] = d_times.dayofyear
    df['month'] = d_times.month
    df['year'] = (d_times.year - 2013)

    df = df.drop(['time'], axis=1) 
    return df
    

def process_one_cell(df_train, df_test, grid_id, th):
    """   
    Classification inside one grid cell.
    """   
    #Working on df_train
    df_cell_train = df_train.loc[df_train.grid_cell.isin(cells_around(grid_id))]
    place_counts = df_cell_train.place_id.value_counts()
    mask = (place_counts[df_cell_train.place_id.values] >= th).values
    df_cell_train = df_cell_train.loc[mask]

    #Working on df_test
    df_cell_test = df_test.loc[df_test.grid_cell == grid_id]
    row_ids = df_cell_test.index
    
    #Preparing data
    le = LabelEncoder()
    y = le.fit_transform(df_cell_train.place_id.values)
    X = df_cell_train.drop(['place_id', 'grid_cell'], axis=1).values.astype(int)
    X_test = df_cell_test.drop(['grid_cell'], axis = 1).values.astype(int)
    
    #Applying the classifier
    clf = RandomForestClassifier(10, n_jobs = -1)
    clf.fit(X, y)
    y_pred = clf.predict_proba(X_test)
    pred_labels = le.inverse_transform(np.argsort(y_pred, axis=1)[:,::-1][:,:3])    
    return pred_labels, row_ids

In [ ]:
print('Loading data ...')
df_train = pd.read_csv('train.csv',
                       usecols=['row_id','x','y','time','place_id'], 
                       index_col = 0)
df_test = pd.read_csv('test.csv',
                      usecols=['row_id','x','y','time'],
                      index_col = 0)

print('Preparing train data')
df_train = prepare_data(df_train, n_cell_x, n_cell_y)
print('Preparing test data')
df_test = prepare_data(df_test, n_cell_x, n_cell_y)

Loading data ...
Preparing train data
Preparing test data


In [ ]:
start_time = time.time()

#Solving classification problems inside each grid cell
th = 5 #Keeping place_ids with more than th samples.
n_cells = n_cell_x*n_cell_y
preds = np.zeros((df_test.shape[0], 3), dtype=int)

for g_id in range(n_cells):
    if g_id % 20 == 0:
        print('iter: %s' %(g_id))
        print("Elapsed time : %s seconds" % (time.time() - start_time))
    
    #Applying classifier to one grid cell
    pred_labels, row_ids = process_one_cell(df_train, df_test, g_id, th)
    #Updating predictions
    preds[row_ids] = pred_labels
print('Generating submission file ...')
#Auxiliary dataframe with the 3 best predictions for each sample
df_aux = pd.DataFrame(preds, dtype=str, columns=['l1', 'l2', 'l3'])  
#Concatenating the 3 predictions for each sample
ds_sub = df_aux.l1.str.cat([df_aux.l2, df_aux.l3], sep=' ')
#Writting to csv
ds_sub.name = 'place_id'
ds_sub.to_csv('knn_submission_' + str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")) + '.csv',
              index=True, header=True, index_label='row_id')         

iter: 0
Elapsed time : 0.000400543212890625 seconds
iter: 20
Elapsed time : 85.64502954483032 seconds


In [5]:
!ls

accuracy exploration.ipynb	     test.csv
knn_blocks-Copy1.ipynb		     train.csv
knn_blocks-Copy2.ipynb		     Untitled1.ipynb
knn_blocks.ipynb		     Untitled.ipynb
knn_submission_2016-06-13-10-50.csv  untitled.txt
rf_blocks.ipynb


In [6]:
!vncserver


New 'X' desktop is ip-172-31-50-254:1

Starting applications specified in /home/ubuntu/.vnc/xstartup
Log file is /home/ubuntu/.vnc/ip-172-31-50-254:1.log

